# Pre-Req:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

In [4]:
#Train Dataset
TrainDataset = pd.read_csv('/content/train.csv')
TrainDataset.head()


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [9]:
TrainDataset.shape

(188533, 13)

In [8]:
#Test Dataset
TestDataset = pd.read_csv('/content/test.csv')
TestDataset.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [7]:
TestDataset.shape

(125690, 12)

In [10]:
#Checking for null values:
TrainDataset.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [11]:
#Checking for null values:
TestDataset.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [49]:
#Train Dataset sort values:

####################################################Vehicle Fuel Types########################################################################
TrainDataset['fuel_type'] = TrainDataset['fuel_type'].replace('-', 'Gasoline') #Changes all values '-' to gasoline
TrainDataset.loc[(TrainDataset['brand'] == 'Tesla') & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')), 'fuel_type'] = 'Electric' #changes empty values for Teslas to electric

###################################################Porsche Taycan Clean#######################################################################
porsche_taycan_mask = (TrainDataset['brand'] == 'Porsche') & TrainDataset['model'].str.contains('Taycan', na=False)

# For Electric engines in Porsche Taycan models
TrainDataset.loc[porsche_taycan_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Electric Motor|Dual AC Electric Motors', na=False), 'fuel_type'] = 'Electric'
# For Gasoline engines in Porsche Taycan models
TrainDataset.loc[porsche_taycan_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Gasoline|Turbo|GDI|V8', na=False), 'fuel_type'] = 'Gasoline'
# For Hybrid engines in Porsche Taycan models
TrainDataset.loc[porsche_taycan_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Hybrid', na=False), 'fuel_type'] = 'Hybrid'

###################################################Nissan Leaf Clean#######################################################################

# Clear the 'fuel_type' column for Nissan Leaf models - All Nissan Lead Vehicles are Electric!
TrainDataset.loc[(TrainDataset['brand'] == 'Nissan') &(TrainDataset['model'].str.contains('Leaf', case=False)),'fuel_type'] = None  # Set 'fuel_type' to None (or NaN) for all Nissan Leaf models

# Replace the cleared 'fuel_type' column with 'Electric' for all Nissan Leaf models
TrainDataset.loc[(TrainDataset['brand'] == 'Nissan') &(TrainDataset['model'].str.contains('Leaf', case=False)),'fuel_type'] = 'Electric'  # Set 'fuel_type' to 'Electric' for all Nissan Leaf models

#######################################################Rivian Clean##################################################################################
# Create a mask for Rivian models
rivian_mask = (TrainDataset['brand'] == 'Rivian')

# For Electric engines in Rivian models
TrainDataset.loc[rivian_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Electric Motor|Dual AC Electric Motors', na=False), 'fuel_type'] = 'Electric'

# For Gasoline engines in Rivian models (if applicable)
TrainDataset.loc[rivian_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Gasoline|Turbo|GDI|V8', na=False), 'fuel_type'] = 'Gasoline'

# For Hybrid engines in Rivian models (if applicable)
TrainDataset.loc[rivian_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Hybrid', na=False), 'fuel_type'] = 'Hybrid'

#######################################################Chevy Bolt Clean##################################################################################
#Create a mask for Chevy Bolt models
chev_bolt_mask = (TrainDataset['brand'] == 'Chevrolet')

# For Electric engines in Rivian models
TrainDataset.loc[chev_bolt_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Electric|Battery Electric', na=False), 'fuel_type'] = 'Electric'

# For Gasoline engines in Rivian models (if applicable)
TrainDataset.loc[chev_bolt_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Fuel', na=False), 'fuel_type'] = 'Gasoline'

#######################################################Ford Mustang Clean##################################################################################
# Create a mask for Ford Mustang models
mustang_mask = (TrainDataset['brand'] == 'Ford') & (TrainDataset['model'].str.contains('Mustang', na=False))

# For Electric engines in Mustang models
TrainDataset.loc[mustang_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Electric Motor|Electric Fuel System|Standard Range Battery|Electric', na=False), 'fuel_type'] = 'Electric'

# For Gasoline engines in Mustang models
TrainDataset.loc[mustang_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Gasoline|Turbo|V8|V6', na=False), 'fuel_type'] = 'Gasoline'

# For Hybrid engines in Mustang models (if applicable, adjust if hybrids are relevant)
TrainDataset.loc[mustang_mask & (TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')) & TrainDataset['engine'].str.contains('Hybrid', na=False), 'fuel_type'] = 'Hybrid'

#######################################################Mercedes Clean##################################################################################
# Create a mask for Mercedes models
mercedes_mask = (TrainDataset['brand'] == 'Mercedes-Benz')

# Filter for rows where 'model', 'model_year', or 'milage' are NaN or empty, and drop them
rows_to_drop = TrainDataset.loc[mercedes_mask & ((TrainDataset['model'].isna() | TrainDataset['model'].eq('')) |(TrainDataset['model_year'].isna() | TrainDataset['model_year'].eq('')) |(TrainDataset['milage'].isna() | TrainDataset['milage'].eq('')))]

# Drop the filtered rows from the original DataFrame
TrainDataset = TrainDataset.drop(rows_to_drop.index)

#########################################################Accident Clean######################################################################
TrainDataset.loc[TrainDataset['accident'].isna() | TrainDataset['accident'].eq(''), 'accident'] = 'No' #Replaces blank cell for accident as no

###########################################################Title Clean######################################################################
TrainDataset.loc[TrainDataset['clean_title'].isna() | TrainDataset['clean_title'].eq(''), 'clean_title'] = 'No' #Replaces blank cell for accident as no

remaining_empty_fuel_type = TrainDataset[TrainDataset['fuel_type'].isna() | TrainDataset['fuel_type'].eq('')]
print("Remaining rows with empty or NaN 'fuel_type':")
print(remaining_empty_fuel_type)

Remaining rows with empty or NaN 'fuel_type':
            id          brand                      model  model_year  milage  \
2181      2181           Ford     F-150 Lightning LARIAT        2020     470   
2394      2394           Ford     F-150 Lightning LARIAT        2017     330   
3286      3286          smart                ForTwo Pure        2008  156875   
3515      3515           Ford        F-150 Lightning XLT        2023     208   
3750      3750  Mercedes-Benz              EQS 450+ Base        2013  174000   
...        ...            ...                        ...         ...     ...   
187854  187854  Mercedes-Benz              EQS 450+ Base        2018   12454   
188005  188005            BMW                   i3 94 Ah        2017   33500   
188201  188201           Audi  Q4 e-tron 50 Premium Plus        2022   51500   
188362  188362     Volkswagen                  e-Golf SE        2021   22185   
188484  188484           Ford              Model X P100D        2018   520

In [50]:
TrainDataset.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,720
engine,0
transmission,0
ext_col,0
int_col,0


# Data Analysis:

# Feature Engineering:

# Data Processing and Results: